# RAG 평가 과정:
<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" height="300px">

## 전체 과정 요약
평가 데이터셋(질문-답변 쌍)을 가지고, 그 중에서 **질문만** RAG 시스템에 주고 답변을 생성합니다. 그 후 시스템이 생성한 답변과 데이터셋의 답변을 비교해서 성능을 평가합니다.

## 단계별 설명

### 1단계: 두 가지 데이터셋 준비

**원본 문서 데이터셋**
- `ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")`으로 불러옵니다.
- 이 문서들이 `RAW_KNOWLEDGE_BASE`에 저장됩니다.
- RAG 시스템이 검색할 정보가 여기 있습니다.

**평가 데이터셋**
- `eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")`으로 불러옵니다.
- 이 데이터셋은 질문과 그에 대한 정답을 포함합니다.
- 시스템 평가에 사용됩니다.

### 2단계: 문서 처리 및 임베딩

**문서 처리**
- `split_documents` 함수가 원본 문서(`RAW_KNOWLEDGE_BASE`)를 작은 청크로 나눕니다.
- 이렇게 하면 질문과 정확히 관련된 부분만 나중에 검색할 수 있습니다.

**임베딩 생성**
- `load_embeddings` 함수가 청크를 OpenAI의 임베딩 모델을 사용해 벡터로 변환합니다.
- 이 벡터들이 `knowledge_index`라는 FAISS 인덱스에 저장됩니다.
- 이제 질문이 주어지면 관련 청크를 빠르게 찾을 수 있습니다.

### 3단계: RAG 시스템으로 답변 생성

**RAG 과정**
- `answer_with_rag` 함수가 다음 작업을 수행합니다:
 1. 평가 데이터셋에서 질문을 가져옵니다.
 2. `knowledge_index.similarity_search(query=question)`을 호출해 이 질문과 관련된 청크를 찾습니다.
 3. 검색된 청크들을 하나의 컨텍스트로 합칩니다.
 4. 이 컨텍스트와 질문을 `RAG_PROMPT_TEMPLATE`에 넣어 프롬프트를 만듭니다.
 5. 이 프롬프트를 GPT-4(`call_llm` 함수)에 보내 답변을 생성합니다.
 6. 생성된 답변과 사용된 청크를 반환합니다.

**테스트 실행**
- `run_rag_tests` 함수는 평가 데이터셋의 모든 질문에 대해 위 과정을 반복합니다:
 1. `answer_with_rag(question, knowledge_index)`를 호출해 답변을 생성합니다.
 2. 질문, 생성된 답변, 데이터셋의 정답, 사용된 청크 등을 `result` 딕셔너리에 저장합니다.
 3. 이 딕셔너리를 `outputs` 리스트에 추가합니다.
 4. 모든 결과를 `output_file` 경로에 JSON 형식으로 저장합니다.

### 4단계: 답변 평가

**평가 과정**
- `evaluate_answers` 함수가 각 결과를 평가합니다:
 1. `output_file`에서 결과를 불러옵니다.
 2. 각 결과에 대해 `evaluation_prompt_template`을 사용해 평가 프롬프트를 만듭니다.
 3. 이 프롬프트에는 질문, 시스템 생성 답변, 데이터셋의 정답이 포함됩니다.
 4. 이 프롬프트를 평가 모델(`eval_chat_model`, GPT-4)에 보내 평가 점수와 피드백을 받습니다.
 5. 점수와 피드백을 원래 결과에 추가해 저장합니다.

### 5단계: 결과 분석

- `outputs` 리스트에 모든 질문, 생성된 답변, 정답, 평가 점수가 저장됩니다.
- 이 데이터를 분석하면 RAG 시스템의 성능을 파악할 수 있습니다.
- 예를 들어, 평균 점수를 계산하거나 다양한 설정을 비교할 수 있습니다.

## 요약

- 원본 문서로 검색 가능한 인덱스(`knowledge_index`)를 만듭니다.
- 평가 데이터셋의 질문을 사용해 RAG 시스템에서 답변을 생성합니다.
- 생성된 답변을 평가 데이터셋의 정답과 비교해 평가합니다.
- 이를 통해 RAG 시스템의 성능을 측정합니다.

In [ ]:
!pip install -q torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [1]:
# IPython 매직은 Python 호환성을 위해 주석 처리합니다.
# %reload_ext autoreload
# %autoreload 2

from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

from huggingface_hub import notebook_login

notebook_login()

### 사용할 데이터 로드

사용할 데이터를 로드합니다. 이를 우리는 `지식 베이스`라고 부르겠습니다.  

실제로 RAG도 하고 평가 데이터도 만들 것입니다.

In [2]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

In [3]:
ds['text'][0]

' Create an Endpoint\n\nAfter your first login, you will be directed to the [Endpoint creation page](https://ui.endpoints.huggingface.co/new). As an example, this guide will go through the steps to deploy [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) for text classification. \n\n## 1. Enter the Hugging Face Repository ID and your desired endpoint name:\n\n<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-documentation/main/assets/1_repository.png" alt="select repository" />\n\n## 2. Select your Cloud Provider and region. Initially, only AWS will be available as a Cloud Provider with the `us-east-1` and `eu-west-1` regions. We will add Azure soon, and if you need to test Endpoints with other Cloud Providers or regions, please let us know.\n\n<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-documentation/main/assets/1_region.png" alt="select region" />\n\n## 3. Define the [S

# 1. 평가 데이터 만들기

합성 평가 데이터셋을 구축합니다.  

LLM에게 해당 문서를 기반으로 질문과 답변(사실 QA 쌍)을 생성하도록 하는 것입니다.

그 후, 생성된 QA 쌍에 대해 품질 필터 역할을 하는 여러 LLM 에이전트를 설정합니다.  

각 에이전트는 특정 결함(예: 근거성, 적절성, 독립성)을 평가합니다.

### 1.1. 소스 문서 준비

다운로드 한 문서를 청킹해야 합니다. 먼저 원활한 청킹을 위해 다운로드 한 문서를 랭체인 형식으로 변환합니다.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

# 다운로드 한 문서를 랭체인 형식으로 변환
langchain_docs = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]

  0%|          | 0/2647 [00:00<?, ?it/s]

첫번째 샘플을 출력해봅시다.

In [5]:
# 다운로드 한 문서를 랭체인 형식으로 변환 후 첫번째 문서 출력
langchain_docs[0]

Document(metadata={'source': 'huggingface/hf-endpoints-documentation/blob/main/docs/source/guides/create_endpoint.mdx'}, page_content=' Create an Endpoint\n\nAfter your first login, you will be directed to the [Endpoint creation page](https://ui.endpoints.huggingface.co/new). As an example, this guide will go through the steps to deploy [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) for text classification. \n\n## 1. Enter the Hugging Face Repository ID and your desired endpoint name:\n\n<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-documentation/main/assets/1_repository.png" alt="select repository" />\n\n## 2. Select your Cloud Provider and region. Initially, only AWS will be available as a Cloud Provider with the `us-east-1` and `eu-west-1` regions. We will add Azure soon, and if you need to test Endpoints with other Cloud Providers or regions, please let us know.\n\n<img src="https://ra

길이 2000으로 다수의 문서로 청킹합니다.

In [6]:
# 각 문서를 최대 길이 2000으로 청킹
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

허깅페이스 문서를 청크사이즈 2000으로 자른 결과 13,841개의 문서를 얻습니다.

In [7]:
print('청킹된 문서의 개수:', len(docs_processed))
print('청킹 후 첫번째 문서:')

# 첫번째 문서가 너무 길어서 길이 500까지만 출력했습니다.
print(docs_processed[0].page_content[:500])

청킹된 문서의 개수: 13841
청킹 후 첫번째 문서:
Create an Endpoint

After your first login, you will be directed to the [Endpoint creation page](https://ui.endpoints.huggingface.co/new). As an example, this guide will go through the steps to deploy [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) for text classification. 

## 1. Enter the Hugging Face Repository ID and your desired endpoint name:

<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-docum


이제 청킹된 위 문서들을 바탕으로 질문과 답변을 만들어보겠습니다.

### 1.2. 질문 생성을 위한 에이전트 설정

QA 쌍 생성을 위해 GPT-4o을 사용합니다.  

다음은 전형적인 GPT-4o API 사용법입니다.

In [8]:
from openai import OpenAI

OPENAI_API_KEY = "여러분의 Key 값"

# OpenAI API 클라이언트 설정
client = OpenAI(api_key=OPENAI_API_KEY)  # 실제 사용 시 자신의 API 키로 대체해야 합니다

def call_llm(prompt: str, model="gpt-4o"):
    """GPT 모델을 호출하여 응답을 받습니다."""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "당신은 질문과 답변을 생성하는 AI 도우미입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
        temperature=0.7
    )
    return response.choices[0].message.content

In [9]:
# GPT-4o API 테스트 호출 결과
call_llm("당신은 누구입니까?")

'저는 인공지능 기반의 도우미로, 정보를 제공하고 질문에 답변하는 역할을 수행합니다. 여러분의 질문에 최대한 정확하고 유용한 답변을 드리기 위해 노력하고 있습니다. 어떻게 도와드릴까요?'

청킹된 문서의 수가 13,841개로 너무 많으므로 랜덤으로 문서 10개만 뽑아서 문서와 질문의 쌍을 만들어봅시다.  

이를 위한 프롬프트입니다.

In [10]:
# 문서가 주어지면 질문과 답변을 작성하시오

QA_generation_prompt = """
당신의 임무는 주어진 문맥을 바탕으로 사실 질문과 그에 대한 답변을 작성하는 것입니다.
사실 질문은 문맥에 있는 특정하고 간결한 사실 정보를 활용해 대답할 수 있어야 합니다.
질문은 사용자가 검색 엔진에 물어볼 수 있는 스타일로 작성되어야 합니다.
즉, 질문에 "해당 본문에 따르면" 또는 "문맥"과 같은 표현이 포함되어서는 안 됩니다.

답변은 다음과 같이 작성하십시오:

Output:::
Factoid question: (사실 질문)
Answer: (사실 질문에 대한 답변)

이제 문맥은 다음과 같습니다.

Context: {context}
Output:::
"""

In [11]:
import random

N_GENERATIONS = 10  # 비용과 시간을 고려하여 10개의 QA 쌍만 생성

print(f"{N_GENERATIONS}개의 QA 쌍 생성 중...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # QA 쌍 생성
    output_QA_couple = call_llm(
        QA_generation_prompt.format(context=sampled_context.page_content)
    )
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "답변이 너무 깁니다"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

10개의 QA 쌍 생성 중...


  0%|          | 0/10 [00:00<?, ?it/s]

10개의 평가 데이터 중 1개만 출력해보았습니다. question과 answer 모두 GPT-4o가 작성한 것입니다.

In [12]:
display(pd.DataFrame(outputs).head(1))

,context,question,answer,source_doc
0,"_Q: Do I have to use the SageMaker Python SDK to use the Hugging Face Deep Learning Containers?_\n\nA: You can use the HF DLC without the SageMaker Python SDK and launch SageMaker Training jobs with other SDKs, such as the [AWS CLI](https://docs.aws.amazon.com/cli/latest/reference/sagemaker/create-training-job.html) or [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_training_job). The DLCs are also available through Amazon ECR and can be pulled and used in any environment of choice.\n\n_Q: Why should I use the Hugging Face Deep Learning Containers?_\n\nA: The DLCs are fully tested, maintained, optimized deep learning environments that require no installation, configuration, or maintenance.\n\n_Q: Why should I use SageMaker Training to train Hugging Face models?_\n\nA: SageMaker Training provides numerous benefits that will boost your productivity with Hugging Face : (1) first it is cost-effective: the training instances live only for the duration of your job and are paid per second. No risk anymore to leave GPU instances up all night: the training cluster stops right at the end of your job! It also supports EC2 Spot capacity, which enables up to 90% cost reduction. (2) SageMaker also comes with a lot of built-in automation that facilitates teamwork and MLOps: training metadata and logs are automatically persisted to a serverless managed metastore, and I/O with S3 (for datasets, checkpoints and model artifacts) is fully managed. Finally, SageMaker also allows to drastically scale up and out: you can launch multiple training jobs in parallel, but also launch large-scale distributed training jobs\n\n_Q: Once I've trained my model with Amazon SageMaker, can I use it with 🤗/Transformers ?_\n\nA: Yes, you can download your trained model from S3 and directly use it with transformers or upload it to the [Hugging Face Model Hub](https://huggingface.co/models).",Can you use a model trained with Amazon SageMaker with the 🤗/Transformers library?\n,"Yes, you can download your trained model from S3 and directly use it with the 🤗/Transformers library or upload it to the Hugging Face Model Hub.",huggingface/blog/blob/main/the-partnership-amazon-sagemaker-and-hugging-face.md


In [13]:
df = pd.DataFrame(outputs)
df[['question', 'answer']]

,question,answer
0,Can you use a model trained with Amazon SageMaker with the 🤗/Transformers library?\n,"Yes, you can download your trained model from S3 and directly use it with the 🤗/Transformers library or upload it to the Hugging Face Model Hub."
1,How can someone contact Hugging Face for inquiries? \n,You can contact Hugging Face at the email address api-enterprise@huggingface.co.
2,What is the prompt used for generating images in the provided script?\n,"The prompt is ""ghibli style, a fantasy landscape with castles""."
3,How can you add a sphere to a scene in the simulate library?\n,"You can add a sphere to a scene by using `scene += sm.Sphere(position=[0, 1, 0], radius=0.2)`."
4,Where can you find the comparison table for ResNet models?\n,https://huggingface.co/timm/seresnextaa101d_32x8d.sw_in12k_ft_in1k_288#model-comparison
5,What does the `inpaint` function use as input besides the prompt and image?\n,The `inpaint` function uses an input mask loaded from a specified path as input besides the prompt and image.
6,What is the purpose of using `config.chunk_size_feed_forward`?\n,The purpose of using `config.chunk_size_feed_forward` is to allow a better trade-off between memory and speed in certain use cases.
7,What is the previous name of the `oneccl_bindings_for_pytorch` module before version 1.12?\n\n,The previous name of the `oneccl_bindings_for_pytorch` module before version 1.12 was `torch_ccl`.
8,Who are some of the authors of the paper that introduced the LeViT model?\n\n,"The authors include Ben Graham, Alaaeldin El-Nouby, Hugo Touvron, Pierre Stock, Armand Joulin, Hervé Jégou, and Matthijs Douze."
9,Who made a fix that prevents the File component from freezing when a large file is uploaded?\n\n,The fix was made by [@aliabid94](https://github.com/aliabid94) in [PR 3191](https://github.com/gradio-app/gradio/pull/3191).


### 1.3. 비평 에이전트 설정

GPT-4o가 생성한 질문은 여러 결함이 있을 수 있습니다.  
이를 위해 검증을 위해 GPT-4o를 다시 호출해볼 것입니다.  

GPT-4o로 데이터를 만들고 GPT-4o로 데이터를 검증하는 것은 흔히 있는 일입니다.

각 질문을 평가하기 위해 아래 기준(근거성, 적절성, 독립성)에 따라 평가하는 에이전트를 설정합니다.

예시:
- **근거성 (Groundedness):** 주어진 문맥에서 질문에 명확하게 답할 수 있는가?
- **적절성 (Relevance):** 질문이 사용자가 실제로 유용하다고 느낄 수 있는가?
- **독립성 (Stand-alone):** 질문이 추가 문맥 없이도 독립적으로 이해될 수 있는가?

또한, LLM에게 먼저 평가 근거를 출력하게 함으로써 점수에 대한 신뢰도를 높입니다.

In [14]:
# 근거성
question_groundedness_critique_prompt = """
주어진 문맥과 질문이 제공됩니다.
당신의 임무는 주어진 문맥에서 해당 질문에 명확하게 대답할 수 있는지를 평가하여 '총 평점'을 제공하는 것입니다.
답변은 1부터 5까지의 척도로 제시하며, 1은 문맥에 전혀 답할 수 없음을, 5는 문맥에서 명확하게 답할 수 있음을 의미합니다.

답변은 다음과 같이 작성하십시오:

Answer:::
Evaluation: (평가 근거를 텍스트로 작성)
Total rating: (1에서 5 사이의 숫자)

반드시 'Evaluation:'과 'Total rating:'의 값을 제공해야 합니다.

이제 질문과 문맥은 다음과 같습니다.

Question: {question}\n
Context: {context}\n
Answer::: """

In [15]:
# 적절성
question_relevance_critique_prompt = """
질문이 주어집니다.
당신의 임무는 해당 질문이 Hugging Face 생태계에서 NLP 애플리케이션을 구축하는 개발자들에게 얼마나 유용한지를 평가하여 '총 평점'을 제공하는 것입니다.
답변은 1부터 5까지의 척도로 제시하며, 1은 전혀 유용하지 않음을, 5는 매우 유용함을 의미합니다.

답변은 다음과 같이 작성하십시오:

Answer:::
Evaluation: (평가 근거를 텍스트로 작성)
Total rating: (1에서 5 사이의 숫자)

반드시 'Evaluation:'과 'Total rating:'의 값을 제공해야 합니다.

이제 질문은 다음과 같습니다.

Question: {question}\n
Answer::: """

In [16]:
# 독립성
question_standalone_critique_prompt = """
질문이 주어집니다.
당신의 임무는 해당 질문이 문맥에 의존하지 않고 독립적으로 이해될 수 있는지를 평가하여 '총 평점'을 제공하는 것입니다.
답변은 1부터 5까지의 척도로 제시하며, 1은 추가 정보 없이는 이해하기 어려움을, 5는 그 자체로 명확함을 의미합니다.
예를 들어, 질문에 "in the context" 또는 "in the document"와 같이 특정 상황을 언급하면 평점은 반드시 1이어야 합니다.

답변은 다음과 같이 작성하십시오:

Answer:::
Evaluation: (평가 근거를 텍스트로 작성)
Total rating: (1에서 5 사이의 숫자)

반드시 'Evaluation:'과 'Total rating:'의 값을 제공해야 합니다.

이제 질문은 다음과 같습니다.

Question: {question}\n
Answer::: """

위의 세 가지 기준으로 10개의 평가 데이터에 대한 적절성을 평가합니다.

In [17]:
print("각 QA 쌍에 대한 평가 생성 중...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            question_groundedness_critique_prompt.format(
                context=output["context"], question=output["question"]
            )
        ),
        "relevance": call_llm(
            question_relevance_critique_prompt.format(question=output["question"])
        ),
        "standalone": call_llm(
            question_standalone_critique_prompt.format(question=output["question"])
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1]
            )
            output.update({
                f"{criterion}_score": score,
                f"{criterion}_eval": eval,
            })
    except Exception as e:
        continue

각 QA 쌍에 대한 평가 생성 중...


  0%|          | 0/10 [00:00<?, ?it/s]

아래는 10개의 QA에 대해서 적절성을 평가한 결과입니다.

In [18]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame.from_dict(outputs)

print("필터링 전 평가 데이터셋:")
display(
    generated_questions[[
        "question",
        "answer",
        "groundedness_score",
        "relevance_score",
        "standalone_score"
    ]]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4) &
    (generated_questions["relevance_score"] >= 4) &
    (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("최종 평가 데이터셋:")
display(
    generated_questions[[
        "question",
        "answer",
        "groundedness_score",
        "relevance_score",
        "standalone_score"
    ]]
)

필터링 전 평가 데이터셋:


,question,answer,groundedness_score,relevance_score,standalone_score
0,Can you use a model trained with Amazon SageMaker with the 🤗/Transformers library?\n,"Yes, you can download your trained model from S3 and directly use it with the 🤗/Transformers library or upload it to the Hugging Face Model Hub.",5,5,5
1,How can someone contact Hugging Face for inquiries? \n,You can contact Hugging Face at the email address api-enterprise@huggingface.co.,5,2,5
2,What is the prompt used for generating images in the provided script?\n,"The prompt is ""ghibli style, a fantasy landscape with castles"".",5,2,1
3,How can you add a sphere to a scene in the simulate library?\n,"You can add a sphere to a scene by using `scene += sm.Sphere(position=[0, 1, 0], radius=0.2)`.",5,1,4
4,Where can you find the comparison table for ResNet models?\n,https://huggingface.co/timm/seresnextaa101d_32x8d.sw_in12k_ft_in1k_288#model-comparison,5,2,2
5,What does the `inpaint` function use as input besides the prompt and image?\n,The `inpaint` function uses an input mask loaded from a specified path as input besides the prompt and image.,5,5,1
6,What is the purpose of using `config.chunk_size_feed_forward`?\n,The purpose of using `config.chunk_size_feed_forward` is to allow a better trade-off between memory and speed in certain use cases.,5,4,2
7,What is the previous name of the `oneccl_bindings_for_pytorch` module before version 1.12?\n\n,The previous name of the `oneccl_bindings_for_pytorch` module before version 1.12 was `torch_ccl`.,5,3,1
8,Who are some of the authors of the paper that introduced the LeViT model?\n\n,"The authors include Ben Graham, Alaaeldin El-Nouby, Hugo Touvron, Pierre Stock, Armand Joulin, Hervé Jégou, and Matthijs Douze.",5,2,1
9,Who made a fix that prevents the File component from freezing when a large file is uploaded?\n\n,The fix was made by [@aliabid94](https://github.com/aliabid94) in [PR 3191](https://github.com/gradio-app/gradio/pull/3191).,5,2,1


최종 평가 데이터셋:


,question,answer,groundedness_score,relevance_score,standalone_score
0,Can you use a model trained with Amazon SageMaker with the 🤗/Transformers library?\n,"Yes, you can download your trained model from S3 and directly use it with the 🤗/Transformers library or upload it to the Hugging Face Model Hub.",5,5,5


위 과정은 평가 데이터를 만드는 과정이었습니다.  
실제로는 이미 평가 데이터를 만들어두었습니다. 이를 로드하여 평가에 사용할 것입니다.

In [19]:
eval_dataset = datasets.Dataset.from_pandas(
    generated_questions, split="train", preserve_index=False
)

## 미리 생성된 평가 데이터셋 불러오기
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

README.md:   0%|          | 0.00/893 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/289k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65 [00:00<?, ? examples/s]

평가 데이터는 총 65건입니다.

In [20]:
eval_dataset

Dataset({
    features: ['context', 'question', 'answer', 'source_doc', 'standalone_score', 'standalone_eval', 'relatedness_score', 'relatedness_eval', 'relevance_score', 'relevance_eval'],
    num_rows: 65
})

# 2. RAG 시스템 구축

위에서 한 것은 평가 데이터를 만든 것이고 지금부터 할 것은 실제 'RAG'입니다.

### 2.1. 벡터 데이터베이스를 위한 청킹 함수 구현

문서를 더 작은 청크로 분할합니다.

우선 원본 문서는 2,647개입니다.

In [21]:
len(ds)

2647

테스트 데이터를 만들 때와 마찬가지로 원활한 청킹을 위해 각 문서를 랭체인 형식으로 변환합니다.

In [32]:
from langchain.docstore.document import Document as LangchainDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]

  0%|          | 0/2647 [00:00<?, ?it/s]

아래는 청킹 함수를 만들어둔 것입니다. 실제로는 임베딩하기 직전에 호출할 것입니다.  
다시 말해 아래의 `load_embeddings()`에서 `split_documents()`를 내부적으로 호출합니다.

In [33]:
def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
) -> List[LangchainDocument]:
    """
    문서를 주어진 청크 크기(`chunk_size`) 단위로 분할하여 문서 리스트를 반환합니다.

    Parameters:
        chunk_size (int): 청크 크기, 즉 문서를 나눌 단위.
        knowledge_base (List[LangchainDocument]): 입력 문서 리스트.

    Returns:
        List[LangchainDocument]: 분할된 문서 리스트 (중복 제거 포함).
    """

    # `RecursiveCharacterTextSplitter`를 사용하여 문서를 청크 단위로 나누는 분할기 생성
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,  # 최대 청크 크기 설정
        chunk_overlap=int(chunk_size / 10),  # 중첩되는 부분 설정 (전체 크기의 10%)
        length_function=len,  # 길이 측정 기준을 단순 문자 수로 설정
        add_start_index=True,  # 분할된 청크의 원본 내 시작 인덱스를 포함
        separators=["\n\n", "\n", ".", " ", ""],  # 분할 기준 (큰 단위부터 작은 단위 순)
    )

    # 분할된 문서를 저장할 리스트
    docs_processed = []

    # 입력된 모든 문서에 대해 청크 분할 수행
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # 중복 제거 (중복된 문서 내용이 포함되지 않도록 처리)
    unique_texts = {}  # 중복 여부를 체크할 딕셔너리
    docs_processed_unique = []

    for doc in docs_processed:
        if doc.page_content not in unique_texts:  # 기존에 없는 내용만 추가
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique  # 중복이 제거된 문서 리스트 반환

### 2.2. Retriever - 임베딩 🗂️

Langchain 벡터 데이터베이스(예: FAISS)를 사용하여 각 문서를 임베딩합니다.

In [41]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os

def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "text-embedding-3-small",
) -> FAISS:
    """
    주어진 임베딩 모델과 문서를 사용하여 FAISS 인덱스를 생성합니다.
    인덱스가 이미 존재하면 이를 로드합니다.

    인자:
        langchain_docs: 문서 리스트
        chunk_size: 청크의 크기
        embedding_model_name: OpenAI 임베딩 모델 이름 (기본값: text-embedding-3-small)

    반환:
        FAISS 인덱스
    """
    # OpenAI 임베딩 모델 로드
    embedding_model = OpenAIEmbeddings(
        model=embedding_model_name,
        openai_api_key=OPENAI_API_KEY,
        disallowed_special=()  # 모든 특수 토큰 허용
    )

    # 임베딩이 디스크에 존재하는지 확인
    index_name = f"index_chunk_{chunk_size}_embeddings_openai-{embedding_model_name}"  # ':' 제거
    index_folder_path = f"./data/indexes/{index_name}/"
    
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
            allow_dangerous_deserialization=True
        )

    else:
        print("인덱스를 찾을 수 없어 생성합니다...")

        # 문서 처리 전에 특수 토큰 제거
        cleaned_docs = []
        for doc in langchain_docs:
            # 특수 토큰 제거 또는 대체
            cleaned_text = doc.page_content.replace("<|endoftext|>", "")
            cleaned_docs.append(
                LangchainDocument(page_content=cleaned_text, metadata=doc.metadata)
            )

        # 문서 청킹 (임베딩 직전에 호출합니다.)
        docs_processed = split_documents(
            chunk_size,
            cleaned_docs
        )
        print('문서 개수:', len(docs_processed))

        # 오픈AI 임베딩 및 벡터 데이터베이스 FAISS에 적재
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

### 2.3. Reader - LLM 💬

실제 RAG를 수행하는 함수 `answer_with_rag()`를 구현합니다.

In [42]:
# 검색 결과와 질문을 바탕으로 답변하라는 RAG 프롬프트
RAG_PROMPT_TEMPLATE = """
문맥에 포함된 정보를 활용하여 질문에 대한 포괄적인 답변을 제공하세요.
질문에만 응답하고, 답변은 간결하고 질문과 관련이 있어야 합니다.
관련이 있을 때는 출처 문서의 번호를 제공하세요.
문맥에서 답변을 도출할 수 없는 경우에는 답변하지 마세요.

Context:
{context}
---
이제 답변해야 할 질문입니다.

Question: {question}
"""

In [43]:
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM
from typing import Optional, List, Tuple

# 검색 결과를 바탕으로 답변하는 함수
def answer_with_rag(
    question: str,
    knowledge_index: VectorStore,
    num_retrieved_docs: int = 10,  # 30에서 10으로 줄임
    num_docs_final: int = 3,       # 7에서 3으로 줄임
) -> Tuple[str, List]:
    """주어진 지식 인덱스를 사용하여 RAG로 질문에 답변합니다. GPT-4를 사용합니다."""
    # Retriever로 문서 검색
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # 텍스트만 유지
    relevant_docs = relevant_docs[:num_docs_final]

    # 각 문서의 길이 제한 (필요한 경우)
    max_doc_length = 5000  # 문서당 최대 문자 수
    truncated_docs = []
    for doc in relevant_docs:
        if len(doc) > max_doc_length:
            truncated_docs.append(doc[:max_doc_length] + "...")
        else:
            truncated_docs.append(doc)

    relevant_docs = truncated_docs

    # 최종 프롬프트 작성
    context = "\n추출된 문서들:\n"
    context += "".join([
        f"문서 {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)
    ])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # GPT-4로 답변 생성
    answer = call_llm(final_prompt, model="gpt-4o")

    return answer, relevant_docs

# 3. RAG 시스템 벤치마킹

평가 데이터셋을 기반으로 RAG 시스템의 출력물을 평가합니다.

평가를 위해 심판 에이전트를 설정합니다.

※ 평가 지표로는 주로 시스템의 신뢰도(정확성)를 측정합니다.

예시:
- 평가 데이터셋은 LLM에 의해 합성 생성되고,
- 이후 [LLM-as-a-judge](https://huggingface.co/papers/2306.05685) 에이전트가 평가를 수행합니다.

In [44]:
from langchain_core.language_models import BaseChatModel

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    knowledge_index: VectorStore,
    output_file: str,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # 사용된 테스트 설정 문서화
):
    """주어진 평가 데이터셋에서 RAG 테스트를 실행하고 결과를 출력 파일에 저장합니다."""
    try:  # 이전 결과가 존재하면 로드
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        # rag 답변하는 함수 호출
        answer, relevant_docs = answer_with_rag(
            question, knowledge_index
        )
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": [doc for doc in relevant_docs],
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f)

In [45]:
EVALUATION_PROMPT = """###작업 설명:
지시사항(내부에 입력이 포함될 수 있음), 평가할 응답, 5점을 받는 기준 답변, 그리고 평가 기준을 나타내는 점수 루브릭이 제공됩니다.
1. 주어진 점수 루브릭을 기준으로 응답의 품질을 평가하는 상세한 피드백을 작성하십시오. (해당 루브릭에 기반하여 평가)
2. 피드백 작성 후, 1부터 5 사이의 정수 점수를 작성하십시오.
3. 출력 형식은 다음과 같이 작성되어야 합니다: "Feedback: {{피드백}} [RESULT] {{1~5 사이의 정수}}"
4. 다른 시작, 종료 문구나 설명을 생성하지 마십시오. 반드시 [RESULT]를 포함시켜야 합니다.

###평가할 지시사항:
{instruction}

###평가할 응답:
{response}

###기준 답변 (5점):
{reference_answer}

###점수 루브릭:
[응답이 기준 답변을 바탕으로 올바르고, 정확하며, 사실적인가?]
Score 1: 응답이 전적으로 틀리거나, 부정확하거나, 사실이 아닙니다.
Score 2: 응답이 대부분 틀리거나, 부정확하거나, 사실이 아닙니다.
Score 3: 응답이 다소 올바르거나, 정확하거나, 사실입니다.
Score 4: 응답이 대부분 올바르고, 정확하며, 사실입니다.
Score 5: 응답이 전적으로 올바르고, 정확하며, 사실입니다.

###피드백:"""

In [46]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage

evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

from langchain.chat_models import ChatOpenAI

eval_chat_model = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=OPENAI_API_KEY)
evaluator_name = "GPT4"

def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """생성된 응답을 평가하고, 결과를 파일에 저장합니다."""
    answers = []
    if os.path.isfile(answer_path):  # 이전 결과가 존재하면 로드
        answers = json.load(open(answer_path, "r"))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)

        # 영어 'Feedback:'으로 분할하여 파싱 (한글 '피드백:' 아님)
        if "Feedback:" in eval_result.content:
            content_after_feedback = eval_result.content.split("Feedback:")[-1]
            feedback, score = [
                item.strip() for item in content_after_feedback.split("[RESULT]")
            ]
        else:
            # 만약 'Feedback:'이 없다면 전체 내용을 사용
            feedback, score = [
                item.strip() for item in eval_result.content.split("[RESULT]")
            ]

        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

🚀 테스트를 실행하고 응답을 평가합시다!

In [48]:
import os

if not os.path.exists("./output"):
    os.mkdir("./output")

for chunk_size in [7000]:  # 필요한 경우 다른 청크 크기 추가
    for embeddings in ["text-embedding-3-small"]:  # OpenAI 임베딩 모델
        # 콜론(`:`)을 `_`로 변경하여 Windows에서도 사용 가능하도록 수정
        settings_name = f"chunk_{chunk_size}_embeddings_openai-{embeddings}_reader-model_gpt-4"
        output_file_name = f"./output/rag_{settings_name}.json"

        print(f"설정 {settings_name}에 대한 평가 실행 중:")

        print("지식 베이스 임베딩 로드 중...")
        knowledge_index = load_embeddings(
            RAW_KNOWLEDGE_BASE,
            chunk_size=chunk_size,
            embedding_model_name=embeddings,
        )

        print("RAG 실행 중...")
        # 실제 rag 실행
        run_rag_tests(
            eval_dataset=eval_dataset,
            knowledge_index=knowledge_index,
            output_file=output_file_name,
            verbose=False,
            test_settings=settings_name,
        )
        print("평가 실행 중...")
        # 동일한 질문(평가 데이터에 존재하는 질문)에 대해서 실제 rag 실행한 결과랑 이상적인 답변이랑 비교
        evaluate_answers(
            output_file_name,
            eval_chat_model,
            evaluator_name,
            evaluation_prompt_template,
        )

설정 chunk_7000_embeddings_openai-text-embedding-3-small_reader-model_gpt-4에 대한 평가 실행 중:
지식 베이스 임베딩 로드 중...
RAG 실행 중...


  0%|          | 0/65 [00:00<?, ?it/s]

평가 실행 중...


  0%|          | 0/65 [00:00<?, ?it/s]

### 결과 확인

In [49]:
import glob

outputs = []
for file in glob.glob("./output/*.json"):
    output = pd.DataFrame(json.load(open(file, "r")))
    output["settings"] = file
    outputs.append(output)
result = pd.concat(outputs)

result["eval_score_GPT4"] = result["eval_score_GPT4"].apply(
    lambda x: int(x) if isinstance(x, str) else 1
)
result["eval_score_GPT4"] = (result["eval_score_GPT4"] - 1) / 4

average_scores = result.groupby("settings")["eval_score_GPT4"].mean()
average_scores.sort_values()

settings
./output\rag_chunk_7000_embeddings_openai-text-embedding-3-small_reader-model_gpt-4.json    0.765385
Name: eval_score_GPT4, dtype: float64

In [51]:
import json

# 파일 경로 지정
file_path = "./output/rag_chunk_7000_embeddings_openai-text-embedding-3-small_reader-model_gpt-4.json"

# 파일 로드 및 내용 확인
with open(file_path, 'r') as f:
    results = json.load(f)

# 결과 개수 확인
print(f"총 {len(results)}개의 평가 결과")

# 처음 5개 결과 확인
for i, result in enumerate(results[:5]):
    print(f"\n{i+1}번째 평가 결과:")
    print(f"질문: {result['question']}")
    print(f"생성된 답변: {result['generated_answer']}")
    print(f"참조 답변: {result['true_answer']}")
    print(f"평가 점수: {result[f'eval_score_{evaluator_name}']}")
    print(f"평가 피드백: {result[f'eval_feedback_{evaluator_name}']}")
    print("="*50)  # 구분선 추가

총 65개의 평가 결과

1번째 평가 결과:
질문: What architecture is the `tokenizers-linux-x64-musl` binary designed for?

생성된 답변: The `tokenizers-linux-x64-musl` binary is designed for the **x86_64-unknown-linux-musl** architecture. [문서 0]
참조 답변: x86_64-unknown-linux-musl
평가 점수: 5
평가 피드백: The response accurately identifies the architecture as x86_64-unknown-linux-musl, which is correct. The additional "[문서 0]" is extraneous but does not detract from the correctness of the answer.

2번째 평가 결과:
질문: What is the purpose of the BLIP-Diffusion model?

생성된 답변: The purpose of the BLIP-Diffusion model is to enable zero-shot subject-driven generation and control-guided zero-shot generation for text-to-image tasks. It supports multimodal control by consuming inputs of subject images and text prompts, and is designed to overcome limitations of existing models, such as lengthy fine-tuning and difficulties in preserving subject fidelity. BLIP-Diffusion introduces a multimodal encoder pre-trained to provide subject rep

참조 답변(reference_answer 또는 true_answer)은 평가를 위해 다운로드한 데이터셋 파일에서 가져온 답변입니다.
코드에서는 eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train") 부분에서 이 데이터셋을 불러왔습니다. 이 데이터셋은 Hugging Face에서 호스팅되는 "m-ric/huggingface_doc_qa_eval"이라는 이름의 데이터셋입니다.

## 예제 결과

여기서는 다양한 RAG 설정(예: 청크 크기, 임베딩 모델, 재정렬 사용 여부 등)을 조정하여 얻은 결과를 확인합니다.

옵션에 따라 성능에 미치는 영향이 다르며, 특히 청크 크기 조정은 큰 영향을 미칠 수 있습니다.

이제 여러분은 이 평가 파이프라인을 바탕으로 다양한 RAG 시스템을 실험해 볼 수 있습니다. 🗺️